In [1]:
from baseline_model import baseline_bicubic
from data_sets_loaders import get_train_val_test_dataloaders
import matplotlib.pyplot as plt
import torch
import numpy as np
from loss_functions import CombinedLoss
from discriminator_model import DiscriminatorModel

cropped_dir = "./cropped_images"
train_dl, val_dl, test_dl = get_train_val_test_dataloaders(100)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
def test_prototype_model(test_dl, upscale_model, discriminator_model, criterion):
    
    test_loss = 0.0
    num_batches = len(test_dl)
    
    with torch.no_grad():
        for inputs_lr, targets_hr in test_dl:
            inputs_lr = inputs_lr.to(device)
            targets_hr = targets_hr.to(device)
            
            generated_hr = upscale_model(inputs_lr)
            generated_outputs = discriminator_model(generated_hr)
            
            loss = criterion(generated_hr, targets_hr, generated_outputs)
            test_loss += loss.item()
    
    avg_test_loss = test_loss / num_batches
    print(f"Average Test Loss: {avg_test_loss}")

In [3]:
upscale_model = torch.load("final_models/model_8_64_10_12_51_g.pth").to(device)
discrim_model = torch.load("final_models/model_8_64_10_12_51_d.pth").to(device)

criterion = CombinedLoss(0.5, 1.0, 2.0, 0.0, 1.0, device)

test_prototype_model(test_dl, upscale_model, discrim_model, criterion)

Average Test Loss: 4.526735591888428


In [4]:
def baseline_model(x):
    return torch.nn.functional.interpolate(x, scale_factor=3, mode='bicubic')

test_prototype_model(test_dl, baseline_model, discrim_model, criterion)

Average Test Loss: 5.4382425308227536
